Как правило, код программ на Python разделяют на функции и классы. Для простоты возьмём программу, состоящую из нескольких функций. При работе этой программы функции будут взаимодействовать друг с другом — не зря же они написаны.

Интерпретатор читает программу сверху вниз — и первым делом выполняет инструкции, которые обнаруживает в глобальной области программы, вне функций или классов.

Когда на очередном шаге встречается вызов какой-то функции, интерпретатор переходит к её выполнению. После того, как вызванная функция закончит работу, — например, дойдёт до инструкции `return`, — выполнение программы продолжится с того места, откуда был совершён переход к этой функции.

События в программе происходят примерно так:

```py
def outer():                              
    print('Выполнение функции outer().')  # Шаг 3.

print('Начало работы программы.')         # Шаг 1.
outer()                                   # Шаг 2.
print('Окончание работы программы.')      # Шаг 4. 
```

При выполнении программы в интерпретаторе создаётся специальный стек, где сохраняется информация о вызовах функций. Он называется «стек вызовов» (англ. call stack).

Именно благодаря стеку вызовов интерпретатор «запоминает» порядок вызовов и определяет, куда он должен вернуться после того, как выполнит очередную функцию.

***
## Как устроен стек вызовов

In [ ]:
def inner():
    pass


def outer():
    inner()


outer()
inner()

Функция `inspect.stack()` возвращает итерируемый объект-генератор, каждый элемент которого описывает отдельный элемент стека вызовов, «шаг» выполняемой программы. Каждый такой элемент содержит довольно большой объём информации.

Этот объект-генератор можно преобразовать в список и напечатать:

In [ ]:
import inspect

# Создадим функцию, которая в нужный момент 
# будет выводить информацию о стеке вызовов:
def print_call_stack():
    # Распаковываем генератор в список и печатаем его:
    print([frame for frame in inspect.stack()])

def outer():
    # Для пробы вызовем здесь информацию о стеке.
    print_call_stack()

outer()

# Будет выведен список из нескольких элементов вида 
# FrameInfo(кортеж_c_информацией_об_элементе_стека):

[FrameInfo(frame=<frame at 0x000002672DF8FB80, file 'C:\\Users\\iliae\\AppData\\Local\\Temp\\ipykernel_16660\\1435700503.py', line 7, code print_call_stack>, filename='C:\\Users\\iliae\\AppData\\Local\\Temp\\ipykernel_16660\\1435700503.py', lineno=7, function='print_call_stack', code_context=['    print([frame for frame in inspect.stack()])\n'], index=0, positions=Positions(lineno=7, end_lineno=7, col_offset=30, end_col_offset=45)), FrameInfo(frame=<frame at 0x000002672E163240, file 'C:\\Users\\iliae\\AppData\\Local\\Temp\\ipykernel_16660\\1435700503.py', line 11, code outer>, filename='C:\\Users\\iliae\\AppData\\Local\\Temp\\ipykernel_16660\\1435700503.py', lineno=11, function='outer', code_context=['    print_call_stack()\n'], index=0, positions=Positions(lineno=11, end_lineno=11, col_offset=4, end_col_offset=22)), FrameInfo(frame=<frame at 0x000002672E163880, file 'C:\\Users\\iliae\\AppData\\Local\\Temp\\ipykernel_16660\\1435700503.py', line 13, code <module>>, filename='C:\\Users\\

> `FrameInfo` — это именованный кортеж, а каждый элемент в этом кортеже описывает один из элементов стека. Все вместе эти элементы содержат информацию из стека вызовов на текущий момент.

***
## Лирическое отступление: именованные кортежи

Это такой тип данных, коллекция, которая обладает всеми свойствами кортежа, но каждому элементу в нём присваивается уникальное имя. К элементам именованного кортежа можно обращаться как к атрибутам объекта. Этот тип данных нужно импортировать из модуля `collections`.

In [2]:
from collections import namedtuple

sputnik = namedtuple('MarsSputnik', ['name', 'area', 'radius', 'gravity'])

phobos = sputnik(name='Phobos', area=1548.3, radius=11.26, gravity=0.0057)
deimos = sputnik(name='Deimos', area=495.15, radius=6.2, gravity=0.003)
print(phobos)
print(deimos)
print(phobos.area)

MarsSputnik(name='Phobos', area=1548.3, radius=11.26, gravity=0.0057)
MarsSputnik(name='Deimos', area=495.15, radius=6.2, gravity=0.003)
1548.3


***
## Распечатываем данные из стека вызовов

Имя четвёртого элемента именованного кортежа — `function`, в нём содержится название функции, выполняемой на этом уровне стека. Оно-то нам и нужно. Переберём содержимое стека при помощи спискового включения, где каждый элемент стека назовём именем `frame`. Тогда получить название функции можно будет через атрибут объекта `frame`: `frame.function`. 

```py
def print_call_stack():
    print([frame.function for frame in inspect.stack()]) 
```

Расставим в коде вызовы функции `print_call_stack()`, она напечатает названия функций, вызванных в определённый момент работы программы. Комментариями отметим порядок, в котором будут печататься названия функций из стека.

Скопируйте программу в свой редактор кода и выполните её. 

In [3]:
import inspect

def print_call_stack():
    # Функция inspect.stack() возвращает объект-генератор.
    # В каждом элементе генератора хранится именованный кортеж.
    # Из этого кортежа берём элемент с названием function: frame.function.
    # В этом элементе хранится название вызванной функции.
    print([frame.function for frame in inspect.stack()])

def inner():
    print_call_stack()  # 4, 7

def outer():
    print_call_stack()  # 3
    inner()
    print_call_stack()  # 5

# А это ответ на вопрос, который сейчас появится.
print([frame.function for frame in inspect.stack()])  # 1
print_call_stack()  # 2
outer()
print_call_stack()  # 6
inner()

['<module>', 'run_code', 'run_ast_nodes', 'run_cell_async', '_pseudo_sync_runner', '_run_cell', 'run_cell', 'run_cell', 'do_execute', 'execute_request', 'execute_request', 'dispatch_shell', 'process_one', 'dispatch_queue', '_run', '_run_once', 'run_forever', 'start', 'start', 'launch_instance', '<module>', '_run_code', '_run_module_as_main']
['print_call_stack', '<module>', 'run_code', 'run_ast_nodes', 'run_cell_async', '_pseudo_sync_runner', '_run_cell', 'run_cell', 'run_cell', 'do_execute', 'execute_request', 'execute_request', 'dispatch_shell', 'process_one', 'dispatch_queue', '_run', '_run_once', 'run_forever', 'start', 'start', 'launch_instance', '<module>', '_run_code', '_run_module_as_main']
['print_call_stack', 'outer', '<module>', 'run_code', 'run_ast_nodes', 'run_cell_async', '_pseudo_sync_runner', '_run_cell', 'run_cell', 'run_cell', 'do_execute', 'execute_request', 'execute_request', 'dispatch_shell', 'process_one', 'dispatch_queue', '_run', '_run_once', 'run_forever', 'sta

В каждой строке напечатаны названия функций в том порядке, как они размещены в стеке вызовов: слева — та функция, что на вершине стека, правее — те, что глубже. 

Если изобразить вызовы в виде вертикальных столбцов, можно увидеть, как нарастает и уменьшается стек при появлении новых вызовов:

![alt text](S10_69_1696832771.png)

Разберём вывод построчно.

1. С первым вызовом уже разобрались: инструкция выполняется прямо из кода модуля — и в стеке лишь одна запись, `<module>`.

2. Второй вывод стека: из кода модуля вызвана функция `print_call_stack()`, она и оказалась на вершине стека. После выполнения этой функции интерпретатор удалит её с вершины стека и вернётся к выполнению кода в `<module>`.

3. Третий вывод стека: из `<module>` вызвана функция `outer()`, а из неё — `print_call_stack()`. И вот все они отображены в стеке, от «верхнего» вызова к «нижнему».

4. Здесь функция `outer()` вызывает `inner()`, а в `inner()` вызывается `print_call_stack()`. И к моменту этого вызова функции `print_call_stack()` в стеке оказываются все функции программы:
`['print_call_stack', 'inner', 'outer', '<module>']`.

5. Функция `inner()` завершила работу — и после завершения она удаляется из стека. На вершине оказывается информация о той функции, которая вызвала `inner()`. Интерпретатор считывает стек и понимает, что следует вернуться к выполнению функции `outer()`, к строке 16. А там — очередной вызов `print_call_stack()`.

6. То же происходит и на следующем шаге: функция `outer()` завершилась, она удаляется из стека; интерпретатор возвращается к выполнению кода из `<module>`, к строке 22.

7. В следующий раз стек печатается из функции `inner()` после того, как она вызвана в строке 23. Из этой функции стек уже печатался на шаге 4, однако теперь стек отличается от предыдущего. В первый раз функция `inner()` была вызвана из `outer()`, и путь до её вызова был длиннее, чем теперь: на этом шаге функция `inner()` вызвана из основного модуля.